In [47]:
import mne
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs


# EEG preprocessing with MNE python

In [48]:
%matplotlib qt

subjects = ['sub05','sub06', 'sub07', 'sub08','sub09','sub10','sub11','sub12','sub13','sub14','sub15','sub16',
'sub18','sub19','sub20','sub21','sub22','sub23', 'sub25', 'sub26', 'sub27', 'sub28', 'sub29', 'sub30', 
'sub31', 'sub32', 'sub33', 'sub34', 'sub35', 'sub36', 'sub37', 'sub38', 'sub39', 'sub40', 'sub41', 'sub42', 'sub43', 
'sub44', 'sub45', 'sub46', 'sub47', 'sub48', 'sub49', 'sub50', 'sub51', 'sub52', 'sub53', 'sub55', 'sub56', 
'sub57', 'sub58', 'sub59', 'sub60', 'sub61', 'sub62',
'sub64', 'sub65', 'sub66', 'sub67', 'sub68', 'sub69', 'sub70', 'sub72']

for sub in subjects:
    
    raw = mne.io.read_raw_fif("../eeg_fieldtrip/data_transformed/{s}-raw.fif".format(s = sub), preload = True)
    
    #Some information about the data
    ch_names = raw.ch_names
    n_chan = len(ch_names) 
    
    #Providing info about the unit 
    for channel in raw.info['chs']:
        channel['unit'] = 107
    
    ##Assigning all channels to eeg for rescaling
    list_name = ch_names
    list_type = ['eeg']*65 + ['stim']
    raw.set_channel_types(dict(zip(list_name, list_type)), on_unit_change = "ignore")
    
    raw.apply_function(lambda x: x * 1e-6)
    
    ##Assigning the correct channel labels 
    list_name = ch_names[:-1]
    list_type = ['eeg']*63+ ['eog']*2 
    raw.set_channel_types(dict(zip(list_name, list_type)), on_unit_change = "ignore")
    

    events = mne.find_events(raw, stim_channel="TRIGGER", shortest_event = 0)
    print(events[:5])  # show the first 5 
    
    ##Cut the data (starting 1 sec before the first trigger, ending 1 sec after the last one)
    raw.crop(tmin = events[0][0]/raw.info["sfreq"]-1, tmax = events[-1][0]/raw.info["sfreq"]+1)
    
    ev_breaks = []
    
    for ev in events:
        if ev[2] == 5 or ev[2] == 6:
            ev_breaks.append(ev)
    
    
    onsets = []
    durations = []
    descriptions = []
    
    
    for i,ev in enumerate(ev_breaks):
        if ev[2] == 5 and i < 22:
            onsets.append(ev[0]/raw.info["sfreq"])
            end_break = ev_breaks[i+1][0]/raw.info["sfreq"]
            dur = end_break - ev[0]/raw.info["sfreq"]
            durations.append(dur)
            descriptions.append("bad break")



    break_annot = mne.Annotations(
        onsets, durations, descriptions, orig_time=raw.info["meas_date"]
    )
    
    raw.set_annotations(break_annot)
    
    raw.set_eeg_reference(ref_channels=['A1', 'A2'])
    raw_filter = raw.copy().filter(l_freq=0.5, h_freq=30, picks = ["eeg", "eog"])
    
    
    #raw_filter.plot()
    badelec = pd.read_excel("preprocessing_clean.xlsx")
    badelecs = badelec[badelec['Subject'] == sub]['Bad electrodes'].str.split(", ").iloc[0]


    for b in badelecs:
        if b.lower() != "none":
            raw_filter.info['bads'].append(b)

    raw_filter.set_montage("standard_1020")

    #ICA
    ica = ICA(n_components=15, max_iter="auto", random_state=97)
    
    


    ica.fit(raw_filter)


    # find which ICs match the EOG pattern
    eog_indices, eog_scores = ica.find_bads_eog(raw_filter, measure = "correlation", threshold = 0.3);
    
    ica.exclude = eog_indices
          

    ica.apply(raw_filter)
    
    interpolated = raw_filter.copy().interpolate_bads()
    
    events = mne.find_events(interpolated, stim_channel="TRIGGER", initial_event = True, shortest_event = 0)
    
    #Downsample to 128 Hz 
    raw_downsampled, events = interpolated.copy().resample(sfreq=128, events = events)
    
    raw_downsampled.save("../eeg_prepro/prepro_{s}-raw.fif".format(s=sub), overwrite = True)



    raw_down_filter = raw_downsampled.copy().filter(l_freq=None, h_freq=15, picks = ["eeg", "eog"])

    
    
    epochs = mne.Epochs(raw_down_filter, events,event_id =1, tmin=0, tmax=6,preload=True, proj = False, baseline = None)
    print(epochs.event_id)
    
    epochs.save("../eeg_prepro/saved_{s}_new-epo.fif".format(s =sub), overwrite=True)

Opening raw data file ../eeg_fieldtrip/data_transformed/sub32_wordlist-raw.fif...
Isotrak not found
    Range : 0 ... 248497 =      0.000 ...   248.497 secs
Ready.
Reading 0 ... 248497  =      0.000 ...   248.497 secs...
Trigger channel TRIGGER has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
79 events found on stim channel TRIGGER
Event IDs: [1 2 3 4]
[[ 2542     0     2]
 [ 3577     0     3]
 [ 9707     0     4]
 [11505     0     1]
 [15045     0     2]]
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower tr

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 5.4s.
Using EOG channels: EOGH, EOGV
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.5

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
